# Cybershuttle Molecular Dynamics SDK

This SDK allows users to define, plan, and execute molecular dynamics experiments with ease. In this notebook, we demonstrate how to authenticate, set up a NAMD experiment, add replicas, create an execution plan, and monitor the execution. The experiment object `exp` is an instance of `cybershuttle.base.Experiment`.

In [1]:
import cybershuttle as cs
from cybershuttle import md

## Authenticate for Remote Execution

To authenticate for remote execution, use the `cs.auth.login()` method. This method will prompt you to enter your credentials and authenticate your session.

In [2]:
cs.auth.login()

User code: UAPQ-RRVR
Please authenticate by visiting: https://auth.cybershuttle.org/realms/10000000/device?user_code=UAPQ-RRVR
Waiting for authorization...
Authorization successful!


## Define a NAMD Experiment

To define a NAMD experiment, we initialize an instance of the `cybershuttle.base.Experiment` class with the required parameters such as the name of the experiment, and the input PDB and PSF files. The experiment object `exp` is created as shown below:

In [3]:
exp = md.NAMD.initialize(
    name="yasith_namd_experiment",
    pdb="input.pdb",
    psf="input.psf",
)

## Add Replicas for NAMD Experiment

To add replicas to the NAMD experiment, we use the `add_replica()` method of the `exp` object. This method adds a replica to the experiment. In this example, we add 4 replicas as shown below:

In [4]:
for _ in range(4):
  exp.add_replica()

## Create Execution Plan

To create an execution plan for the experiment, use the `plan()` method of the `exp` object. This method will generate a plan for the experiment, which can then be described using the `describe()` method. The code to create and describe the plan is shown below:

In [5]:
plan = exp.plan()  # this will create a plan for the experiment
plan.describe()  # this will describe the plan

Task(app_id=namd, inputs={'pdb': 'input.pdb', 'psf': 'input.psf'}, runtime=Local(args={}))
Task(app_id=namd, inputs={'pdb': 'input.pdb', 'psf': 'input.psf'}, runtime=Local(args={}))
Task(app_id=namd, inputs={'pdb': 'input.pdb', 'psf': 'input.psf'}, runtime=Local(args={}))
Task(app_id=namd, inputs={'pdb': 'input.pdb', 'psf': 'input.psf'}, runtime=Local(args={}))


In [6]:
plan.save("plan.json")
cs.plan.load("plan.json").describe()  # this will load the plan and describe it

Task(app_id=namd, inputs={'pdb': 'input.pdb', 'psf': 'input.psf'}, runtime=Local(args={}))
Task(app_id=namd, inputs={'pdb': 'input.pdb', 'psf': 'input.psf'}, runtime=Local(args={}))
Task(app_id=namd, inputs={'pdb': 'input.pdb', 'psf': 'input.psf'}, runtime=Local(args={}))
Task(app_id=namd, inputs={'pdb': 'input.pdb', 'psf': 'input.psf'}, runtime=Local(args={}))


## Execute the Plan

In [7]:
plan.execute()  # this will execute the plan (will ask for user confirmation)

Preparing execution plan...
Task(app_id=namd, inputs={'pdb': 'input.pdb', 'psf': 'input.psf'}, runtime=Local(args={}))
Task(app_id=namd, inputs={'pdb': 'input.pdb', 'psf': 'input.psf'}, runtime=Local(args={}))
Task(app_id=namd, inputs={'pdb': 'input.pdb', 'psf': 'input.psf'}, runtime=Local(args={}))
Task(app_id=namd, inputs={'pdb': 'input.pdb', 'psf': 'input.psf'}, runtime=Local(args={}))
Confirming execution plan...
Starting tasks...
Copying data to compute resource:  {'pdb': 'input.pdb', 'psf': 'input.psf'}
Executing app_id=namd on Local: {}
Assigned exec_id=7633d319-52a1-40a8-b10e-f197f1382759 to task
Copying data to compute resource:  {'pdb': 'input.pdb', 'psf': 'input.psf'}
Executing app_id=namd on Local: {}
Assigned exec_id=65f337ed-143e-45ec-bc10-61baf809682a to task
Copying data to compute resource:  {'pdb': 'input.pdb', 'psf': 'input.psf'}
Executing app_id=namd on Local: {}
Assigned exec_id=7728baef-02d2-4b36-ae0d-257f0854ac46 to task
Copying data to compute resource:  {'pdb':

## Option A - Wait for Completion

In [8]:
plan.wait_for_completion(check_every_n_mins=10)  # this will block execution until the plan is completed - don't poll

Waiting for completion


## Option B - Terminate Execution

In [9]:
plan.terminate()

Terminating plan


## Option C - Monitor Files During Execution

Displaying the status and files generated by each replica (task)

In [10]:
for task in plan.tasks:
    status = task.status()
    files = task.files()
    print(status, files)

COMPLETED []
COMPLETED []
COMPLETED []
COMPLETED []


Displaying the intermediate results generated by each replica (task)

In [11]:
from matplotlib import pyplot as plt
import pandas as pd

for index, task in enumerate(plan.tasks):

    @cs.task_context(task)
    def visualize():
        data = pd.read_csv("data.csv")
        plt.figure(figsize=(8, 6))
        plt.plot(data["x"], data["y"], marker="o", linestyle="-", linewidth=2, markersize=6)
        plt.title(f"Plot for Replica {index} of {len(plan.tasks)}")

    visualize()

TypeError: visualize() got an unexpected keyword argument 'pdb'